In [ ]:
### Datset replace with own

import pandas as pd
from pprint import pprint

list_ds =[

{"instruction":"What is the acquistion date", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"28/Jul/2022"},
{"instruction":"What is the acquistion date", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups by 01MARCH2025", "output":"01MARCH2025"},

{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
]

pprint(list_ds)



# Install Flash Attention 2 for softcapping support
import torch


### FINE TUNE

from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)
#### ACTUAL TRAINING
trainer_stats = trainer.train()


##### RESULTS

FastLanguageModel.for_inference(model)

mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""

mna_news_instruction1 = """What is the acquistion date. Answer precisely"""
mna_news_instruction2 = """Which company is the Acquirer. . Answer precisely """

inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction1,
       mna_news_input,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]
### Output : January 05, 2022<eos>
### Above is Awesome exactly what we wanted

inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction2,
       mna_news_input,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]
### Output : HCL Technologies<eos>